### Create a counting model

notebook to create a Poisson counting model

We observe **nobs** events and we expect **b** background events with error **sigmab**

In [1]:
using namespace RooFit;
using namespace RooStats;

int nobs; double b; double sigmab; 

In [2]:
nobs= 3;                // number of observed events
b = 1;                 // number of background even
sigmab = 0.2;          // relative uncertainty in b

In [3]:
RooWorkspace w("w");
   
// make Poisson model * Gaussian constraint
w.factory("sum:nexp(s[3,0,15],b[1,0,10])");
// Poisson of (n | s+b)
w.factory("Poisson:pdf(nobs[0,50],nexp)");
w.factory("Gaussian:constraint(b0[0,10],b,sigmab[1])");
w.factory("PROD:model(pdf,constraint)");


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create the **ModelConfig** class

In [4]:
RooStats::ModelConfig mc("ModelConfig",&w);
mc.SetPdf(*w.pdf("model"));
mc.SetParametersOfInterest(*w.var("s"));
mc.SetObservables(*w.var("nobs"));
mc.SetNuisanceParameters(*w.var("b"));

// these are needed for the hypothesis tests
mc.SetSnapshot(*w.var("s"));
mc.SetGlobalObservables(*w.var("b0"));

mc.Print();
// import model in the workspace 
w.import(mc);


=== Using the following for ModelConfig ===
Observables:             RooArgSet:: = (nobs)
Parameters of Interest:  RooArgSet:: = (s)
Nuisance Parameters:     RooArgSet:: = (b)
Global Observables:      RooArgSet:: = (b0)
PDF:                     RooProdPdf::model[ pdf * constraint ] = 4.45985e-16
Snapshot:                
  1) 0x7f97778614c0 RooRealVar:: s = 3  L(0 - 15)  "s"



Set the model parameters ( number of background events and its uncertainty) 

In [5]:
w.var("b0")->setVal(b);
w.var("b0")->setConstant(true); // needed for being treated as global observables
w.var("sigmab")->setVal(sigmab*b);    

Make data set with the given number of observed events

In [6]:
RooDataSet data("data","", *w.var("nobs"));
w.var("nobs")->setVal(3);
data.add(*w.var("nobs") );
// import data set in workspace 
w.import(data);

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing dataset data


Print workspace and save it in a file

In [7]:
w.Print();
w.writeToFile("CountingModel.root", true);


RooWorkspace(w) w contents

variables
---------
(b,b0,nobs,s,sigmab)

p.d.f.s
-------
RooGaussian::constraint[ x=b0 mean=b sigma=sigmab ] = 1
RooProdPdf::model[ pdf * constraint ] = 0.195367
RooPoisson::pdf[ x=nobs mean=nexp ] = 0.195367

functions
--------
RooAddition::nexp[ s + b ] = 4

datasets
--------
RooDataSet::data(nobs)

parameter snapshots
-------------------
ModelConfig__snapshot = (s=3)

named sets
----------
ModelConfig_GlobalObservables:(b0)
ModelConfig_NuisParams:(b)
ModelConfig_Observables:(nobs)
ModelConfig_POI:(s)
ModelConfig__snapshot:(s)

generic objects
---------------
RooStats::ModelConfig::ModelConfig

